In [6]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

In [7]:
# read disease identifiers table
table = pd.read_csv('../results/All_DOIDs_CONCEPTs_diseases.csv', sep=',', header=0, index_col=0)
table

,disease id,concepts,doid,diseases
0,dz:72,C0013264,DOID:11723,Duchenne muscular dystrophy
1,dz:72,C0026850,DOID:9884,muscular dystrophy
2,dz:72,C0026848,DOID:423,myopathy
3,dz:72,C0026857,DOID:17,musculoskeletal system disease
4,dz:72,C0012634,DOID:4,disease
...,...,...,...,...
2865,dz:655,C0861352,DOID:3010,lobular neoplasia
2866,dz:655,C1332315,DOID:8428,breast apocrine carcinoma in situ
2867,dz:655,C0154061,DOID:8826,colon carcinoma in situ
2868,dz:655,C0851140,DOID:8991,cervix uteri carcinoma in situ


In [8]:
# create a dictionary of concept Ids and disease names
keys = table['concepts']
values = table['diseases']
table_dic = {keys[i]: values[i] for i in range(len(keys))}

In [9]:
# read the file of disease-concept ids 
dz_con = pd.read_csv('../results/ancestors_offsprings.csv', sep=',', header=0, index_col=0)
dz_con= dz_con[['dz_id', 'concept']]
dz_con= dz_con.reset_index(drop=True)
dz_con

,dz_id,concept
0,dz:328,C0013264
1,dz:328,C0026848
2,dz:328,C0026850
3,dz:328,C0012634
4,dz:328,C0026857
...,...,...
17564,dz:655,C0861352
17565,dz:655,C0154092
17566,dz:655,C0851140
17567,dz:655,C0154069


In [10]:
# Indication matrix (broad id of drugs vs concept ids of diseases)
brd_conc = pd.read_csv('../results/brd_conc.csv', sep=',', header=0, index_col=0)
brd_conc

,C0000810,C0000814,C0001126,C0001127,C0001144,C0001206,C0001207,C0001261,C0001263,C0001264,...,C3897537,C3897731,C3898069,C3898092,C3898222,C3898877,C3900099,C4038730,C4048328,C4083212
BRD-ID,,,,,,,,,,,,,,,,,,,,,
BRD-A00100033,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRD-A00546892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRD-A00758722,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRD-A00827783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
BRD-A00993607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BRD-K99984802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRD-M47793593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRD-M98649031,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# open the correlation matrix (broad id of drugs vs disease ids of diseases)
similarity = pd.read_csv('../results/brd_dz_corr.csv', sep=',', header=0, index_col=0)
similarity 

,dz-1000,dz-1001,dz-1002,dz-1003,dz-1004,dz-1005,dz-1011,dz-1012,dz-102,dz-1021,...,dz-979,dz-98,dz-980,dz-981,dz-982,dz-983,dz-984,dz-985,dz-990,dz-991
BRD-K06674495,0.066670,0.118966,0.125683,0.057989,0.122721,0.064734,-0.009684,0.101766,-0.007885,0.022460,...,0.019637,0.091178,0.072941,-0.065441,-0.072216,0.105552,0.029268,0.080375,0.068325,0.062684
BRD-K13005598,-0.013880,0.035840,0.013460,-0.027457,0.019176,-0.035669,0.032979,-0.043762,0.026163,0.033194,...,-0.047346,-0.025270,0.004441,-0.009225,-0.000506,0.008497,-0.015131,0.003364,-0.044631,-0.116571
BRD-K53414658,-0.025052,-0.031393,-0.073677,-0.064345,0.057462,-0.057956,0.037644,0.026204,0.011634,0.089243,...,-0.131232,-0.150644,-0.031836,-0.031777,-0.019102,-0.003967,-0.014603,0.026543,-0.026507,-0.056030
BRD-K09485525,0.067112,0.057495,-0.071661,-0.087242,-0.055037,-0.099037,-0.002947,0.108484,0.016799,-0.001486,...,-0.274513,0.056862,-0.039405,-0.126530,-0.148273,-0.128911,-0.022638,-0.066035,-0.174796,-0.190250
BRD-K02950022,0.016955,0.021994,0.029253,-0.051965,-0.010874,-0.026066,-0.028676,0.031235,0.006593,0.106154,...,-0.135260,0.018354,0.004145,-0.054970,-0.102150,-0.020334,0.002388,-0.061010,-0.080655,-0.099899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BRD-K06234293,-0.029219,-0.044929,-0.105232,-0.131461,-0.049543,-0.053180,-0.043853,0.006456,-0.047078,0.018269,...,0.047330,-0.103736,0.077848,0.039492,0.048736,0.103372,-0.113326,0.002839,-0.029638,-0.029233
BRD-K64925568,0.021497,0.022991,0.040036,-0.025479,-0.015981,-0.033822,0.007910,0.055046,-0.019045,0.074883,...,-0.374318,0.046411,-0.062879,-0.038719,-0.020123,-0.158803,0.026248,-0.018008,-0.140115,-0.298888
BRD-A13133631,-0.096410,-0.032140,0.041751,0.047089,0.085241,0.010350,0.022568,0.003022,0.016884,0.122504,...,-0.061172,-0.035743,-0.071672,-0.019662,0.039339,0.053417,0.092866,-0.001433,0.006978,0.038669
BRD-K84658672,0.056069,-0.012038,-0.076591,-0.114969,-0.035062,-0.066394,0.007560,0.039698,0.007977,-0.080815,...,-0.082534,-0.022123,-0.029680,-0.039606,-0.030218,-0.017106,0.009238,-0.008238,-0.054368,-0.081949


In [14]:
# adjusting the colum names (disease ids)
good_columns = []
for c in similarity.columns:
    good_columns.append(c.replace('-',':'))

In [15]:
similarity.columns = good_columns

In [17]:
# open table of concept Ids with corresponding AUC scores.
conc_AUC_table = pd.read_csv('../results/conc_AUC_table.csv', sep=',', header=0, index_col=0)
conc_AUC_table

,conc_ids,AUC
0,C0013264,0.393740
1,C0026857,0.457104
2,C0878544,0.395567
3,C0018799,0.566260
4,C0004623,0.770322
...,...,...
219,C0040028,0.030679
220,C1292778,0.436630
221,C0005779,0.971512
222,C0154301,0.535242


In [18]:
concept_unique = []
concept_unique = dz_con['concept'].unique()

In [19]:
## Applying Man-whitney U test between diseases in the corr matrix and indication matrix
## and record the resulted P-value from each comparison in the table 
df_app = pd.DataFrame()
df_nonapp = pd.DataFrame()
p_value_list = []
disease = []
for i in concept_unique:
    all_conc_df = dz_con.loc[dz_con['concept'] == i]
    all_conc_df['Indicator']= 1
    all_conc_df = pd.pivot_table(all_conc_df, index='concept', columns='dz_id', values='Indicator')
    all_conc_df[all_conc_df.isna()] = 0
    all_conc_df = all_conc_df.astype(int)
    common_db = list(set(brd_conc.columns) & set(all_conc_df.index))
    if len(common_db)>0:
        conc_df_brd_dz = brd_conc[common_db] @ all_conc_df.loc[common_db]
        common_brd = list(set(conc_df_brd_dz.index) & set(similarity.index))
        if len(common_brd)>0:
            common_dz = list(set(conc_df_brd_dz.columns) & set(similarity.columns))
            if len(common_dz)>0:
                p_table = pd.DataFrame()
                conc_ed =  conc_df_brd_dz.loc[common_brd,common_dz]
                sim_df =  similarity.loc[common_brd,common_dz]
                conc_ed[conc_ed>0] = 1
                y_tr = conc_ed.values.reshape((1, -1))[0]
                y_pr = sim_df .values.reshape((1,-1))[0]
                p_table['pred'] = y_tr
                p_table['corr'] = y_pr
                df_app = p_table.loc[p_table['pred'] == 1]
                df_nonapp = p_table.loc[p_table['pred'] == 0]
                stat, p = mannwhitneyu(df_app['corr'], df_nonapp['corr'])
                if y_tr.sum()>0:  
                    p_value_list.append(p)
                    disease.append(table_dic[i])
conc_AUC_table['p_value'] = p_value_list
conc_AUC_table['diseases'] = disease

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

C:\Users\Saba\anaconda3\lib\site-packages\scipy\stats\stats.py:7028: RuntimeWarning: divide by zero encountered in double_scalars
  z = (bigu - meanrank) / sd
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
C:\Users\Saba\anaconda3\lib\site-packages\scipy\stats\stats.py:7028: RuntimeWarning: divide by zero encountered in double_scalars
  z = (bigu - meanrank) / sd
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

...

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_conc_df['Indicator']= 1
<ipython-input-19-1a5811981594>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [20]:
conc_AUC_table

,conc_ids,AUC,p_value,diseases
0,C0013264,0.393740,0.092313,Duchenne muscular dystrophy
1,C0026857,0.457104,0.000188,musculoskeletal system disease
2,C0878544,0.395567,0.153212,cardiomyopathy
3,C0018799,0.566260,0.245642,heart disease
4,C0004623,0.770322,0.001544,bacterial infectious disease
...,...,...,...,...
219,C0040028,0.030679,0.052260,essential thrombocythemia
220,C1292778,0.436630,0.312343,myeloproliferative neoplasm
221,C0005779,0.971512,0.051455,blood coagulation disease
222,C0154301,0.535242,0.365529,acquired thrombocytopenia


In [15]:
conc_AUC_table.to_csv('../results/p_value_table.csv')